# Backward selection

In [1]:
HOUSING_PATH = "data/housing.csv"

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
housing = pd.read_csv(HOUSING_PATH)
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
class PipelineFriendlyLabelBinarizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None,**fit_params):
        return self
    def transform(self, X):
        return LabelBinarizer().fit(X).transform(X)

In [7]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer

num_attribs = strat_train_set.columns.drop(["ocean_proximity", "median_house_value"])
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', PipelineFriendlyLabelBinarizer()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [8]:
X_train = strat_train_set.drop("median_house_value", axis=1)
y_train = strat_train_set["median_house_value"].copy()

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

In [9]:
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)

In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression().fit(X_train_prepared, y_train)
y_pred = lin_reg.predict(X_test_prepared)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

/Users/tural/.virtualenvs/P3/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


66911.980708575458

## Выбор фич

In [11]:
columns = [
    'longitude', 'latitude', 'housing_median_age', 'total_rooms', 
    'total_bedrooms', 'population', 'households', 'median_income',
    'rooms_per_household', 'population_per_household', 'bedrooms_per_room',
    'lt_1H_OCEAN', 'INLAND', 'ISLAND', 'NEAR_BAY', 'NEAR_OCEAN', 'median_house_value'
]

train_df = pd.DataFrame(np.c_[X_train_prepared, y_train], columns=columns)
test_df = pd.DataFrame(np.c_[X_test_prepared, y_test], columns=columns)

In [12]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,population_per_household,bedrooms_per_room,lt_1H_OCEAN,INLAND,ISLAND,NEAR_BAY,NEAR_OCEAN,median_house_value
0,-1.156043,0.771950,0.743331,-0.493234,-0.445438,-0.636211,-0.420698,-0.614937,-0.312055,-0.086499,0.155318,1.0,0.0,0.0,0.0,0.0,286600.0
1,-1.176025,0.659695,-1.165317,-0.908967,-1.036928,-0.998331,-1.022227,1.336459,0.217683,-0.033534,-0.836289,1.0,0.0,0.0,0.0,0.0,340600.0
2,1.186849,-1.342183,0.186642,-0.313660,-0.153345,-0.433639,-0.093318,-0.532046,-0.465315,-0.092405,0.422200,0.0,0.0,0.0,0.0,1.0,196900.0
3,-0.017068,0.313576,-0.290520,-0.362762,-0.396756,0.036041,-0.383436,-1.045566,-0.079661,0.089736,-0.196453,0.0,1.0,0.0,0.0,0.0,46300.0
4,0.492474,-0.659299,-0.926736,1.856193,2.412211,2.724154,2.570975,-0.441437,-0.357834,-0.004194,0.269928,1.0,0.0,0.0,0.0,0.0,254500.0


In [13]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/tural/.virtualenvs/P3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Итерация 0

In [14]:
_iteration_cols = train_df.columns.drop('median_house_value')

In [15]:
def get_formula(_cols):
    return 'median_house_value ~ ' + " + ".join(_cols)

In [16]:
get_formula(_iteration_cols)

'median_house_value ~ longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income + rooms_per_household + population_per_household + bedrooms_per_room + lt_1H_OCEAN + INLAND + ISLAND + NEAR_BAY + NEAR_OCEAN'

In [17]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68634.5400488
R^2: 0.648097455502


In [18]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.985e+16,2.78e+16,1.076,0.282,-2.46e+16,8.43e+16
longitude,-5.555e+04,2295.187,-24.202,0.000,-6e+04,-5.1e+04
latitude,-5.66e+04,2417.955,-23.410,0.000,-6.13e+04,-5.19e+04
housing_median_age,1.372e+04,616.660,22.250,0.000,1.25e+04,1.49e+04
total_rooms,-1937.9780,2190.851,-0.885,0.376,-6232.282,2356.326
total_bedrooms,7334.2437,3199.489,2.292,0.022,1062.900,1.36e+04
population,-4.57e+04,1374.831,-33.242,0.000,-4.84e+04,-4.3e+04
households,4.546e+04,3145.483,14.453,0.000,3.93e+04,5.16e+04
median_income,7.472e+04,755.430,98.905,0.000,7.32e+04,7.62e+04
rooms_per_household,6601.8937,675.865,9.768,0.000,5277.125,7926.662


### Итерация 2

In [19]:
# выкиним total_rooms так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('total_rooms')

In [20]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68631.9239797
R^2: 0.648124281174


In [21]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,3.801e+16,3.55e+16,1.072,0.284,-3.15e+16,1.08e+17
longitude,-5.561e+04,2293.723,-24.246,0.000,-6.01e+04,-5.11e+04
latitude,-5.672e+04,2414.664,-23.491,0.000,-6.15e+04,-5.2e+04
housing_median_age,1.376e+04,616.120,22.328,0.000,1.25e+04,1.5e+04
total_bedrooms,5930.2787,2784.972,2.129,0.033,471.433,1.14e+04
population,-4.603e+04,1323.141,-34.791,0.000,-4.86e+04,-4.34e+04
households,4.532e+04,3141.269,14.427,0.000,3.92e+04,5.15e+04
median_income,7.451e+04,714.343,104.299,0.000,7.31e+04,7.59e+04
rooms_per_household,6519.3589,668.806,9.748,0.000,5208.426,7830.292
population_per_household,1076.1516,551.820,1.950,0.051,-5.475,2157.778


### Итерация 3

In [22]:
# выкиним NEAR_OCEAN так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('NEAR_OCEAN')

In [23]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68629.8181151
R^2: 0.648145874369


In [24]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.223e+05,1667.166,133.313,0.000,2.19e+05,2.26e+05
longitude,-5.571e+04,2291.741,-24.311,0.000,-6.02e+04,-5.12e+04
latitude,-5.68e+04,2413.529,-23.534,0.000,-6.15e+04,-5.21e+04
housing_median_age,1.375e+04,616.086,22.322,0.000,1.25e+04,1.5e+04
total_bedrooms,5940.7106,2784.869,2.133,0.033,482.066,1.14e+04
population,-4.604e+04,1323.095,-34.795,0.000,-4.86e+04,-4.34e+04
households,4.532e+04,3141.172,14.427,0.000,3.92e+04,5.15e+04
median_income,7.45e+04,714.286,104.297,0.000,7.31e+04,7.59e+04
rooms_per_household,6519.6056,668.786,9.748,0.000,5208.713,7830.498
population_per_household,1076.6370,551.803,1.951,0.051,-4.956,2158.230


### Итерация 4

In [25]:
# выкиним lt_1H_OCEAN так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('lt_1H_OCEAN')

In [26]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68637.2454412
R^2: 0.648069712816


In [27]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.196e+05,876.457,250.525,0.000,2.18e+05,2.21e+05
longitude,-5.667e+04,2235.502,-25.350,0.000,-6.11e+04,-5.23e+04
latitude,-5.775e+04,2361.016,-24.459,0.000,-6.24e+04,-5.31e+04
housing_median_age,1.372e+04,615.841,22.273,0.000,1.25e+04,1.49e+04
total_bedrooms,6137.4265,2783.140,2.205,0.027,682.172,1.16e+04
population,-4.624e+04,1318.805,-35.063,0.000,-4.88e+04,-4.37e+04
households,4.529e+04,3141.372,14.416,0.000,3.91e+04,5.14e+04
median_income,7.438e+04,711.542,104.531,0.000,7.3e+04,7.58e+04
rooms_per_household,6566.4694,668.378,9.824,0.000,5256.376,7876.562
population_per_household,1099.9052,551.708,1.994,0.046,18.497,2181.313


### Итерация 5

In [28]:
# выкиним population_per_household так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('population_per_household')

In [29]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68645.5127449
R^2: 0.647984928243


In [30]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.196e+05,876.534,250.498,0.000,2.18e+05,2.21e+05
longitude,-5.662e+04,2235.564,-25.327,0.000,-6.1e+04,-5.22e+04
latitude,-5.766e+04,2360.793,-24.423,0.000,-6.23e+04,-5.3e+04
housing_median_age,1.375e+04,615.684,22.331,0.000,1.25e+04,1.5e+04
total_bedrooms,6261.4385,2782.696,2.250,0.024,807.055,1.17e+04
population,-4.56e+04,1279.507,-35.641,0.000,-4.81e+04,-4.31e+04
households,4.457e+04,3120.870,14.280,0.000,3.84e+04,5.07e+04
median_income,7.446e+04,710.465,104.803,0.000,7.31e+04,7.59e+04
rooms_per_household,6535.1012,668.253,9.779,0.000,5225.253,7844.949
bedrooms_per_room,9567.1946,704.116,13.588,0.000,8187.051,1.09e+04


### Итерация 6

In [31]:
# выкиним total_bedrooms так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('total_bedrooms')

In [32]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68656.0446777
R^2: 0.647876904183


In [33]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.195e+05,876.371,250.489,0.000,2.18e+05,2.21e+05
longitude,-5.624e+04,2229.567,-25.226,0.000,-6.06e+04,-5.19e+04
latitude,-5.736e+04,2357.482,-24.333,0.000,-6.2e+04,-5.27e+04
housing_median_age,1.365e+04,614.294,22.226,0.000,1.24e+04,1.49e+04
population,-4.593e+04,1271.527,-36.120,0.000,-4.84e+04,-4.34e+04
households,5.097e+04,1278.335,39.874,0.000,4.85e+04,5.35e+04
median_income,7.44e+04,710.020,104.781,0.000,7.3e+04,7.58e+04
rooms_per_household,7179.5128,603.849,11.890,0.000,5995.904,8363.122
bedrooms_per_room,1.001e+04,676.430,14.795,0.000,8681.879,1.13e+04
INLAND,-3.75e+04,1934.988,-19.381,0.000,-4.13e+04,-3.37e+04


### Итерация 7

In [34]:
# выкиним NEAR_BAY так как у него наибольшоее p-value
_iteration_cols = _iteration_cols.drop('NEAR_BAY')

In [35]:
lm = smf.ols(get_formula(_iteration_cols), train_df).fit()
rss = np.sum(lm.resid ** 2)
rmse = np.sqrt(rss / len(lm.resid))
print("RMSE:", rmse)
print("R^2:", lm.rsquared)

RMSE: 68669.7157389
R^2: 0.647736657954


In [36]:
lm_res = lm.summary()
lm_res.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.187e+05,809.450,270.133,0.000,2.17e+05,2.2e+05
longitude,-5.553e+04,2212.506,-25.098,0.000,-5.99e+04,-5.12e+04
latitude,-5.744e+04,2357.711,-24.361,0.000,-6.21e+04,-5.28e+04
housing_median_age,1.336e+04,603.727,22.131,0.000,1.22e+04,1.45e+04
population,-4.574e+04,1269.687,-36.026,0.000,-4.82e+04,-4.33e+04
households,5.071e+04,1274.310,39.791,0.000,4.82e+04,5.32e+04
median_income,7.439e+04,710.130,104.751,0.000,7.3e+04,7.58e+04
rooms_per_household,7150.3119,603.844,11.841,0.000,5966.713,8333.910
bedrooms_per_room,1.001e+04,676.542,14.799,0.000,8685.847,1.13e+04
INLAND,-3.666e+04,1906.996,-19.223,0.000,-4.04e+04,-3.29e+04


## Посмотрим на результат

In [37]:
sub_X_train_prepared = train_df[_iteration_cols].values
sub_X_test_prepared = test_df[_iteration_cols].values

In [38]:
# было 66911.980708575458

In [39]:
lin_reg = LinearRegression().fit(sub_X_train_prepared, y_train)
y_pred = lin_reg.predict(sub_X_test_prepared)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

66961.151959308263